In [4]:
#环境设置
import os
import re
import torch
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.backends.cudnn.benchmark=True
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import cv2
from skimage import io
from PIL import Image
import torchvision
from torch import nn
from torch import optim
import torchvision.utils as vutils
from torchvision import transforms
from torchvision import models as M
from torchvision import datasets as dest
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchinfo import summary

import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd
import datetime
from time import time
import gc

from sklearn.model_selection import train_test_split

manualSeed=1412
torch.manual_seed(manualSeed)
random.seed(manualSeed)
np.random.seed(manualSeed)

In [5]:
torch.cuda.is_available()

True

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [8]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 2070 SUPER (UUID: GPU-ddd6b802-f52d-1fc7-5bf2-dc2c56c5fd41)


In [9]:
torch.cuda.memory_reserved()

0

In [10]:
torch.cuda.memory_allocated()

0

# 【案例】Kaggle医学影像识别

### 4.5 染色标准化进阶：Autoencoders、Unet与Pix2Pix

#### 4.5.1 自动编码器家族Autoencoders

在之前的课程当中，我们已经学习了从0生成图像的生成对抗网络架构GAN，也学习了基于某些有效信息来生成图像的infoGAN与cGAN。依赖于创新的对抗式学习、独特的数据流、独特的损失函数等精妙设计，对抗架构很快就在图像生成和图像处理的领域开辟了一块天地。但在2014年，除了对抗学习这一重大的进步之外，图像生成领域还诞生了著名的“从图像到图像”的生成模型：变分自动编码器（Variational Autoencoders，VAE）。这一架构是从自动编码器（Autoencoders）衍生而来，它与线性层所构成的GAN在能力上较为相似，经过适当训练之后它可以生成难辨真假的手写数字与人脸数据。

然而，变分自动编码器的“声势”却不像生成对抗网络那样浩大，其一是因为生成对抗网络有巨大的研究潜力，其二是由于研究者们早已对于类似于变分自动编码器的各类编码器架构非常熟悉了。自动编码器（Autoencoders）是深度学习领域经典的无监督网络之一，因其作为生成模型的种种表现而闻名，但其诞生最初的目的并不是为了“图像生成”，而是为了实现各种目的下的“数据表示”。

**数据表示（Data Representation）是使用另一种形式呈现原始数据的方法**，这一技术也被称为隐式表示（latent Representation）或者转码（coding）。举例说明：

- 原始数据[2,4,6,8,10]
> 我们可以使用文字**以2开头、以10结尾的偶数列**来表示该原始数据，也可以使用**\[x,2x,3x,4x,5x]且x=2**来表示该原始数据。此时，文字“以2开头、以10结尾的偶数列”和“[x,2x,3x,4x,5x]且x=2”就是原始数据的2种数据表示。

- 原始数据为["苹果","梨","百香果"]
> 我们可以使用序列 **\[0,1,2]** 来表示该原始数据，也可以使用**水果**这一概括性的词汇来表示原始数据。此时，[0,1,2]和“水果”都是原始数据的数据表示。

很显然，一个数据的数据表示不是唯一的，且这种表示可以是精确的、也可以是有些模糊的，甚至可以看起来与原始数据毫不相关，但无论如何，数据表示的结果必须携带原始数据上大部分的信息。**广义地来说，只要数据B是以另一种形式呈现数据A、并且数据B上携带数据A大部分的信息，我们就可以说B是A的数据表示**。同时，“另一种形式”既可以是文字-数字这样不同类别的数据之间的形式差异，也可以是数字-数字这样相同类别、但不同大小、不同数量的数据之间的形式差异。在实际计算当中，当数据B是数据A的数据表示时，数据B常常是从数据A总结出的规律、或直接在数据A上计算得出的新数据。

不难发现，根据数据表示的广义定义，我们非常熟悉的数据编码（独热编码、顺序编码等操作）、特征提取、升维降维、Embedding等方法都可以被囊括到数据表示领域当中。在这一领域当中，**使用机器学习或深度学习手段令算法自己求解出数据表示结果的领域被称之为表征学习（Representation Learning）**。自动编码器正是表征学习领域极具特色的代表架构，因此自动编码器常常被用于降维、特征提取这些“将原始数据转化、提炼为另外的表现形式”的领域。毫无疑问的，为了实现数据表示的功能，自动编码器能够“接收数据A，并输出另一种形式的数据B”，因此自动编码器是为“生产新数据”而生的架构。

那自动编码器是一种怎样的架构呢？首先，最为简单的自动编码器是由线性层构成的，它看起来就像是一个普通的深度神经网络DNN，只不过包含两大其他架构不具备的特征：

1. **输出层的神经元数量往往与输入层的神经元数量一致**
> 在有监督神经网络当中，输出层上的神经元数量必须根据标签的类别来决定，但对于自动编码器这样的无监督算法而言，输出层上的神经元数量理论上是可以自由定义的。在惯例当中，输出层的神经元数量往往与输入层的神经元数量一致，这样的结构能够保证输出与原始数据结构一致、但具体数值不同的数据表示，也有利于检验输出数据是否携带了原始数据大量甚至全部的信息。例如，当输入数据和输出数据为尺寸一致的图像时，只要将图像进行可视化，肉眼也可辨别出输出图像是否携带了足够的原始图像的信息。

2. **网络架构往往呈对称性，且中间结构简单、两边结构复杂**
> 由于输入数据与输出数据结构基本一致，因此自动编码器的网络结构不会呈现出与深度神经网络或卷积神经网络相似的“由大到小”形态，反而会呈现出“大-小-大”的对称性。以下面的架构为例，网络最左侧为输入层，最右侧为输出层，输入层与输出层上的神经元个数都为10个，而输入层与输出层之间是先将神经元数量压缩、再将神经元数量提升的结构。这类先压缩、再提升的结构被称为“瓶颈结构”，与残差网络中的瓶颈结构有较大区别。

![](https://miro.medium.com/max/1400/1*C6Z6i1_2EJn13jVEsAOkRQ.png)

对比之下，其他架构一般是从左至右由大到小：

![](https://www.topbots.com/wp-content/uploads/2020/02/LeNet5_800px_web.jpg)

无论是在DNN还是CNN架构当中，将数据量逐渐压缩的过程都是在进行“信息提纯”（特征提取），对自动编码器而言也是如此。在上面的架构图中，从输入层开始压缩数据、直至架构中心的部分被称为**编码器Encoder**，从原始数据中提纯出的信息被称之为**编码Code**或**隐式表示Latent Representation**，从编码开始拓展数据、直至输出层的部分被称为**解码器Decoder**，解码器的输出一般被称为**重构数据Reconstructions**。值得注意的是，编码器的最后一层被称为**编码层Coding Layer**，编码层的结构被称为**隐式空间Latent Space**，当编码层上的神经元数量越多，隐式空间就越大，编码结果Code上所携带的原始数据的信息也会越多。

![](https://miro.medium.com/max/1400/1*C6Z6i1_2EJn13jVEsAOkRQ.png)

任意自动编码器都是由编码器和解码器共同构成的，在这一架构中，编码器的职责是从原始数据中提取必要的信息，而解码器的职责是将提取出的信息还原为原来的结构，二者共同工作，便能够将原始数据表示成相同结构、但不同数值的另一组数据。值得注意的是，在自动编码器的架构中，编码器的结果Code是原始数据的数据表示，而解码器输出的结果也是原始数据的数据表示。

很显然，如果你对深度神经网络足够熟悉，那自动编码器的架构很容易理解。同时，只需要将编码器中的线性层替换成卷积层、解码器中的线性层替换成转置卷积层，我们就可以轻易地实现深度卷积自动编码器的架构：

![](https://i2.wp.com/sefiks.com/wp-content/uploads/2018/03/convolutional-autoencoder.png?resize=1024%2C342&ssl=1)

这一架构也非常简单，只要了解卷积神经网络及其拓展网络，就可以很快速地实现上述架构。然而，看似简单的架构却让人满腹困惑，甚至有许多“细思极恐”的问题没能解决，比如说：

1. **作为一个无监督架构，自动编码器如何训练？**

> 就像任意有监督网络一样地训练。自动编码器的各个层上拥有需要求解的参数$w$（权重），它也拥有损失函数，它的训练目标是最小化损失函数以求解全部的参数$w$。它可以正向传播、反向传播，可以使用一切我们在有监督网络中学过的技巧进行优化和训练。

2. **自动编码器没有使用标签，那它的损失函数是什么？**

> 神经网络架构的损失函数总是与该架构存在的目的息息相关。自动编码器的根本目标是实现有效的数据表示，即根据原始数据A、产出与原始数据A形式不一致、但携带大量原始数据信息的新数据B。在新生成数据B的数据量等于或小于原始数据A的数据量的情况下，B与A携带的信息相似性越高，代表数据表示的程度越深、数据表示的效率越高。因此，A与B之间的信息差异越小，自动编码器的效果就越好，相反，A与B之间的信息差异越大，就说明自动编码器越糟糕。因此，**自动编码器追求的是原始数据A与新生成数据B之间的信息相似性，自动编码器的迭代方向就是令A与B的信息差异更小的方向**。<br><br>
> 那如何衡量数据A与B之间的差异呢？最常使用的当然是MAE或MSE这些距离类衡量指标，除此之外，在统计学上专门用于衡量不同数据的分布差异的指标，如KL散度、交叉熵等也可以被使用。这些指标可以轻易地衡量两组数据之间的差异，只不过在大多数有监督场景下他们衡量的是真实值与预测值之间的差异，而在自动编码器的场景下他们衡量的是原始数据A与输出数据B之间的差异。
![](http://miro.medium.com/max/1400/1*nGFy96r63GwSE_EsJDLMDw.png)
>在自动编码器的整个损失函数中，这一部分被称为**重构损失 Reconstruction Loss**，它用以衡量自动编码器在将数据转变为另一种形态过程时产生的信息损失。通常来说，自动编码器会在重构损失的基础上，再根据具体架构的需要添加一些惩罚项，共同构成损失函数。

3. **如果衡量指标是原始数据A与新生成数据B之间的信息差，神经网络难道不会直接复制原始数据吗？**

> 大部分时候，自动编码器的输入就是原始数据A，因此如果不对自动编码器设置任何的阻碍，它有可能会直接复制原始数据，因为这样得到的信息差(损失函数值)是最小的。但实际操作起来却不是那么容易——毕竟神经网络中含有加和、激活函数、卷积、拉平等复杂数学运算，要保证所有的样本在同一套参数$w$下都能够将原始数据直接复制到输出层，不是那么容易的一件事。同时，所有自动编码器中的数据必须被压缩到很小的维度后再放大，这要求网络必须舍弃一些信息，这样网络要想直接把信息复制到输出层就更困难了。<br><br>
> 当然，只要网络足够深、训练次数足够多，神经网络也是可能实现“忽略中间复杂数学运算、直接把输入结果复制到输出层”这样的神迹的，因此我们在实际构建自动编码器的时候，**我们需要人为地为算法增加一些“干扰”，以逼迫算法去学习高于当前具体数据的规律**，而不同的干扰方式构成了不同的自动编码器。

4. **都有什么样的干扰方法？都有哪些典型的自动编码器？**

> - **降噪自动编码器（Denoising Autoencoder）**<br><br>
> 为了给模型增加“难度”，降噪自动编码器在输入数据中添加噪音以干扰架构学习，同时也考虑对网络添加Dropout等结构来阻断信息的直接传播。这种情况下，自动编码器的输入就不再是原始数据A了，而是原始数据A与噪音融合后的结果。
![](https://www.oreilly.com/library/view/deep-learning-by/9781788399906/assets/5100d26b-63c5-4c69-93cd-4fb8df5ddcb2.png)<br>

In [ ]:
矩阵 - 0越多，越稀疏
神经网络 - 活跃神经元越少，神经网络越稀疏

> - **稀疏自动编码器（Sparse Autoencoder）**<br><br>
> 稀疏自动编码器通过控制稀疏性来提升编码器“提纯数据”的能力，逼迫编码器舍弃一些信息。例如，在编码层的100个神经元中，如果只有10个神经元上能够输出非0数字，而其他神经元上都是0或者非常接近于0的数字，那编码器必将会尽全力将最有用的信息集中在10个能够表达信息的神经元上，而舍弃一切不必要的信息。<br><br>
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/34_.PNG)
> 怎么实现这一效果呢？只要编码层上不够稀疏，我们就可以在损失函数中施加惩罚，以此来逼迫架构向着稀疏的方向学习。在稀疏自动编码器中，有两种手段可以实现上述效果：<br><br>
> (1) 先使用sigmoid函数将编码层上所有神经元上的输出控制在(0,1)之内，再人为设定阈值$\alpha$（往往是0.1或更小的数字），并规定输出值大于$\alpha$的神经元为活跃神经元（active neurons），然后将所有活跃神经元上的输出绝对值加和后放入损失函数作为惩罚项。即，网络中输出大数字的神经元越多，网络受到的惩罚越大，以此逼迫网络将数字聚集在少量的神经元上。<br><br>
> (2) 同样使用sigmoid函数将神经元的输出值控制在(0,1)之间，再规定编码层的目标稀疏程度为$p$（往往是0.1或更小的数字），计算编码层上所有神经元的输出值的均值与p之间的MSE，放入损失函数；或使用编码层上所有神经元的输出与p计算KL散度，并将KL散度值放入损失函数。即，编码层上输出值与p之间的差异越大，网络受到的惩罚就越大，以此逼迫编码层输出接近目标稀疏程度$p$的值。<br>

> - **变分自动编码器（Variational Autoencoder，VAE）**<br><br>
> 不同于其他自动编码器、也不同于任意的其他深度网络架构，变分自动编码器斩断了神经网络中惯例的“从输入到输出”的数据流，以此杜绝了信息直接被复制到输出层的可能性。对任何真实样本$i$而言，变分自动编码器的编码-解码步骤如下：<br>
<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/01/ML1533-image003.jpg" height = 300><br>
> 1. 首先，变分自动编码器中的编码器会尽量将样本$i$所携带的所有特征信息$X_i$的分布转码成**类高斯分布**$d_i$，该类高斯分布虽然是以高斯分布为目标来编码的，但它一般无法被编码成完美的高斯分布，这一分布被称之为实际分布`Actual Distribution`或隐式分布`Latent Distribution`<br><br>
> 2. 编码器需要输出该类高斯分布的均值$\mu_i$与标准差$\sigma_i$作为编码结果Code<br><br>
> 3. 以均值$\mu_i$与标准差$\sigma_i$为基础构建完美的高斯分布$D_i$，这一分布被称之为目标分布`Target Distribution`<br><br>
> 4. 从完美的高斯分布$D_i$中随机抽取出**一个数值**$z_i$，将该数值输入解码器<br><br>
> 5. 解码器基于$z_i$进行解码，并最终输出与样本$i$的原始特征结构一致的数据，作为变分自动编码器的输出<br><br>
> 不难发现，由于**变分自动编码器根本不会将原始数据的编码结果Code直接传给解码器**，因此输入数据被直接复制到输出的可能性几乎已经不存在了。同时值得注意的是，一个样本只会指向一个$z_i$，因此解码器的输入的结构应该是n_samples个$z_i$，这与生成对抗网络当中直接输入随机数的做法类似。

5. **自动编码器能够输出与原始数据结构一致、数值不一致、但又携带原始数据信息的新数据，但这有什么用呢？**

> 在大部分机器学习/深度学习落地应用场景当中（例如，图像识别、用户流失等），我们会直接使用有监督架构输出的预测标签，但无监督算法输出的并不是标签，因此其具体落地方式要灵活很多、甚至还可以将网络架构拆开应用。对于自动编码器而言，一旦完成训练，则可以在下列场景中实现落地应用：<br><br>
![](https://i2.wp.com/sefiks.com/wp-content/uploads/2018/03/convolutional-autoencoder.png?resize=1024%2C342&ssl=1)
> - **通用降维**：<br><br>
> 舍弃解码器、只将编码器部分拆出来用于降维，在这种情况下，编码器的输出Code就是应用时的唯一输出。由于训练的时候要求解码器的输出B与原始数据A高度相似，因此训练完毕之后我们就能够保证编码结果一定是包含大量原始数据信息的。这样得出的降维结果一定能够最大程度保证信息不丢失。数据被降维之后，如果用于数据储存，就可以大大减少储存空间，如果用于算法训练，就可以大大提升算法效率，甚至可能提升算法的预测结果。<br><br>
> 但需要注意的是，虽然特征衍生、升维属于数据表示的范围，但我们却很少使用自动编码器去对陌生数据进行升维，因为自动编码器的训练方向是“先提炼信息，再将提炼后的信息复原”，而不是“从较少的信息中解读出较多信息、提升数据质量”。如果你对特征工程有所了解，相信你能够理解其中的区别。

> - **提升数据储存效率**：<br><br>
> 使用编码器将数据降维后进行储存/展示，需要使用数据时再使用解码器对数据进行复原，这一流程已经在谷歌云盘的**轻量储存**模块下进行实验。当我们将图像上传云盘时，编码器会瞬间对图像进行压缩，并只保存原始图像1/4的像素量，当我们在本地获取云盘图像时，解码器又会瞬间将图像还原会原本的像素量，以此节约云盘中的储存空间。<br><br>
> 相似的技术还可以被用于**查看原图**功能，当在社交软件上收到图像时，用户只会收到经过编码器压缩后的1/8或1/10略缩图，只有当用户要求查看原图时，我们才会再使用解码器将图像复原。当然，在某些场景下也可能是直接将原始图像传输到本地。
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/35.PNG)

> - **数据生成**：<br><br>
> 在自动编码器家族当中，大部分架构都只能够实现“数据改良”，即在输入数据上进行修改，真正能够实现数据生成的其实只有变分自动编码器这一类，因此**在整个自动编码器家族中，只有变分自动编码器可以被称之为“生成模型”**。<br><br>
> 在变分自动编码器中，编码器的输入是原始数据$X$，但解码器的输入的却是随机抽样出的$z$，因此当变分自动编码器被训练好之后，我们可以只取架构中的解码器来使用：只要对解码器输入任意的随机数$z$，解码器就可以生成仿佛从原始数据$X$中抽样出来的数据，如此就能够实现图像生成。许多论文已经证明，变分自动编码器的生成能力足以与一些生成对抗网络分庭抗礼，但这一架构在生成领域的局限也很明显：与GAN一样，变分自动编码器能够获得的信息只有随机数$z$，因此在面临复杂数据时架构会显得有些弱小。同时，由于该架构在训练和预测过程中都有随机性，也不太适合于想要生成指定数据的场景。但无论如何，变分自动编码器的数据生成能力是不可忽视的。
> <img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/01/ML1533-image003.jpg" height = 300>

> - **数据修复/通用图像处理（降噪、图像复原、上色、智能抠图、去水印等）**：<br><br>
> 在降噪自动编码器中，架构接收带有噪音的图像，但却输出不带噪音的图像，说明在输入数据中加入“噪音”的行为赋予了架构降噪的能力。这一能力可以被用于视频、图像、语音或其他普通数据的降噪场景。同理，如果我们在图像数据中随机挖掉一块（为这些像素点都填上0像素、或255像素值），在经过适当训练之后，自动编码器也可以为我们将图像复原。因此我们可以根据自己的需求训练各种各样能够处理“残缺”数据的自动编码器，因此自动编码器也可以被用于各式各样图像的修复和补全了。<br><br>
> <center><img src="http://raw.githubusercontent.com/leehomyc/Faster-High-Res-Neural-Inpainting/master/images/teaser.png" alt="drawing" width="500"/><center><br>

到这里，我相信你已经了解自动编码器这一族算法了，相信在自动编码器的众多应用当中，图像修复和补全的部分是最令人印象深刻的。遗憾的是，大部分时候自动编码器做出的图像修复和补全无法做到特别完美，常常会有“毛边”和明显的痕迹存在。下图是基于生成对抗网络的复杂架构在图像修复方面的成果，我们可以看出明显的对比：

<center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S0141938221000391-gr1.jpg" alt="drawing" width="500"/><center>

很显然，与当前已经非常成熟的SOTA模型们比起来，自动编码器在图像处理方面的能量比较有限，**只要更换输入数据就可以训练自动编码器来达成迥然不同的目的（例如，图像上色和图像降噪），这是一个非常振奋人心、并且非常有启发性的消息**，这一点使得自动编码器拥有了巨大的应用潜力，并且可以作为输出数据、生成数据的架构存在于各类顶级模型当中。

为什么自动编码器的潜力如此之大呢？举例说明，在降噪自动编码器中，我们实际上准备了真实数据A和带噪音的数据A'两组数据，在训练过程中，我们让架构从A'生成数据B，并且一直致力于缩小B与A之间的差异。在适当训练后，我们得到了能够通过A'生成B，且令B与A高度相似的架构。这一过程看似平常，但其实**等同于一个有监督的过程**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/38.PNG)

不难发现，自动编码器中损失函数衡量A与B之间的差异，而有监督算法当中损失函数衡量预测标签yhat与真实标签y之间的差异，无论我们面对的任务是给图像上色、还是给图像补全、还是其他任务，只要认为A是真实标签、A'是特征矩阵，那自动编码器就可以被当成一个有监督算法使用。如果想要为黑白图像上色，你应该准备彩色图像作为原始数据A，黑白图像作为输入数据A'，如果你想要将冬天的照片变成夏天，那你应该准备夏天的照片作为原始数据A，冬天的照片作为原始数据A'。至此，专用于“数据表示”的无监督算法就成为了可以“依葫芦画瓢”的有监督算法了，真是妙哉。

在所有把自动编码器当做有监督算法使用的场景中，图像分割（Image Segmentation）是最受关注也最为重要的场景之一，而在这一场景中最负盛名的架构之一就是基于自动编码器改进而来的有监督方法Unet。下一节，我们来认识一下经典图像分割架构Unet。

#### 4.5.2 变分自动编码器 Variational Autoencoder (VAE)

变分自动编码器是Encoder-Decoder家族最负盛名、也最受关注的编码器类型，这不仅仅是因为变分自动编码器是整个Encoder-Decoder家族中为数不多的生成模型，也因为这一类架构背后有及其复杂的数学原理与实现代码，这令研究者们欢呼雀跃，同时让众多深度学习新手苦不堪言。变分自动编码器背后的数学原理十分艰深，在课程有限的时间内难以完全展开描述，因此在今天的课程当中，我们的目标是让大家深度了解变分自动编码器的实际运行过程。

首先，我们可以简单概括一下变分自动编码器的实际架构：与普通自动编码器一样，变分自动编码器有编码器Encoder与解码器Decoder两大部分组成，原始图像从编码器输入，经编码器后形成隐式表示（Latent Representation），之后隐式表示被输入到解码器、再复原回原始输入的结构。然而，与普通Autoencoders不同的是，**变分自用编码器的Encoder与Decoder在数据流上并不是相连的，我们不会直接将Encoder编码后的结果传递给Decoder**，具体流程如下：

> 1. 首先，变分自动编码器中的编码器会尽量将样本$i$所携带的所有特征信息$X_i$的分布转码成**类高斯分布**$d_i$，该类高斯分布虽然是以高斯分布为目标来编码的，但它一般无法被编码成完美的高斯分布，这一分布被称之为实际分布`Actual Distribution`或隐式分布`Latent Distribution`<br><br>
> 2. 编码器需要输出该类高斯分布的均值$\mu_i$与标准差$\sigma_i$作为编码结果Code<br><br>
> 3. 以均值$\mu_i$与标准差$\sigma_i$为基础构建完美的高斯分布$D_i$，这一分布被称之为目标分布`Target Distribution`<br><br>
> 4. 从完美的高斯分布$D_i$中随机抽取出**一个数值**$z_i$，将该数值输入解码器<br><br>
> 5. 解码器基于$z_i$进行解码，并最终输出与样本$i$的原始特征结构一致的数据，作为变分自动编码器的输出<br>

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/01/ML1533-image003.jpg" height = 200>

根据以上流程，变分自动编码器的Encoder在输出时，并不会直接输出原始数据的隐式表示，而是会输出从原始数据提炼出的均值$\mu$和标准差$\sigma$。之后，我们需要建立均值为$\mu$、标准差为$\sigma$的正态分布，并从该正态分布中抽样出隐式表示z，再将隐式表示z输入到Decoder中进行解码。对隐式表示z而言，它传递给Decoder的就不是原始数据的信息，而只是与原始数据同均值、同标准差的分布中的信息了。

这个流程描述起来似乎并不复杂，但实际的数据流却没有这么简单。在这里，我为大家梳理了三个需要梳理的重点细节：

> **1. 在实际运算当中，Encoder不会先输出$d_i$、再根据$d_i$计算出$\mu_i$和$\sigma_i$，而是直接输出满足类正态分布要求的$\mu_i$和**$\sigma_i$。即，编码过程中产生的均值$\mu$与标准差$\sigma$并不是通过均值或标准差的定义计算出来的，而是直接从Encoder网络中输出的值。<br><br>
> **2. 为了保证Encoder输出的均值和标准差满足类正态分布，变分自动编码器在损失函数中设置了惩罚项——一旦均值和标准差所反馈的分布`Actual Distribution`与完美的正态分布`Target Distribution`有差异，变分自动编码器就会受到惩罚**。故而在实际的算法运行流程中，Encoder负责输出均值和标准差，损失函数保证均值和标准差是符合某种类正态分布的，这就等价于Encoder将原始数据向类正态分布的方向编码、再输出该类正态分布的均值与标准差。<br><br>
> **3. 由于存在随机抽样过程，架构中的数据流是断裂的，因此反向传播无法进行**，因此我们需要独特的重参数技巧来完成变分自动编码器的反向传播。

这三个细节让整个数据过程变得有些复杂，接下来我们来抽丝剥茧地讲解整个数据过程：

- **变分自动编码器的数据流**

让我们以单一样本和最简单的情况为例，详细讲述一下该过程中的各个细节。

首先，假设存在m个样本，5个特征，数据结构为(m,5)。同时，假设Encoder与Decoder中都只有2层带3个神经元的线性层，且**每个样本只生成一个均值与一个标准差**，则转化流程如下所示：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/46.png)

此时，任意样本$i$经过Encoder后会输出一个均值$\mu_i$与一个标准差$\sigma_i$，可以认为样本$i$上所有的特征信息都被认为属于分布$N\sim(\mu_i,\sigma_i)$，故而此时$\mu_i$与$\sigma_i$已经携带了样本$i$上尽量多的信息。此时，整个Encoder的输出是形如(m,1)的均值向量$\boldsymbol{\mu}$和标准差向量$\boldsymbol{\sigma}$。针对这两个向量中的每一组$(\mu_i,\sigma_i)$，我们都可以生成相应的完美正态分布。有了完美正态分布之后，我们可以从**每个正态分布中随机抽选一个数字**，并按样本排列顺序拼凑在一起，构成形如(m,1)的$\boldsymbol{z}$向量。此时，$\boldsymbol{z}$向量再输入Decoder，Decoder的输入层就只能有1个神经元，因为$z$只有一列。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/47.png)

**注意，一组均值和标准差只能生成一个正态分布，而一个正态分布中只能抽选一个数字，这是变分自编码器抽样的基本规则**。因此，如果每个样本经过Encoder后只输出了一组均值和标准差，那$\boldsymbol{z}$自然只能有一列，隐式空间的结构只能为(m,1)。此时，$\boldsymbol{z}$就是我们抽出的隐式表示，所以Decoder解码的信息都来源于抽样出的样本向量$\boldsymbol{z}$。

大家或许会感觉到奇怪——难道一个样本还可以有多组均值和标准差吗？当然可以。之前我们强调过，在变分自动编码器的流程当中，均值和标准差都不是通过他们的数学定义计算出来的，而是通过Encoder提炼出来的。这就是说**当前的均值和标准差不是真实数据的统计量，而是通过Encoder推断出的、当前样本数据可能服从的任意分布中的属性**。我们不可能知道当前样本服从的真实分布的状态，因此这一推断过程自然可以根据不同的规则（Encoder中不同的权重）得出不同的结果。

例如，我们可以令Encoder的输出层存在3个神经元，这样Encoder就会对每一个样本推断出三对不同的均值和标准差。这个行为相当于对样本数据所属的原始分布进行估计，但给出了三个可能的答案。因此现在，在每个样本下，我们就可以基于三个均值和标准差的组合生成三个不同的正态分布了。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/48.png)

每个样本对应了3个正态分布，而3个正态分布中可以分别抽取出三个数字$z$，此时每个隐式表示$\boldsymbol{z}$就是一个形如(m,3)的矩阵。将这一矩阵放入Decoder，则Decoder的输入层也需要有三个神经元。此时，我们的隐式空间就是(m,3)。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/49.png)

对任意的自动编码器而言，隐式空间越大，隐式表示$\boldsymbol{z}$所携带的信息自然也会越多，自动编码器的表现就可能变得更好，因此在实际使用变分自动编码器的过程中，一个样本上至少都会生成10~100组均值和标准差，隐式表示$\boldsymbol{z}$的结构一般也是较高维的矩阵。

现在，回过头来看变分自动编码器的结构，是不是感觉清晰了许多？很明显，在变分自动编码器中，输入层的信息并没有直接传到输出层，而是在中间进行了截断，原始样本的信息被更换成了“与原始样本处于相同分布中的样本的信息”。由于该过程的存在，变分自动编码器有如下的两个特点：

1. **无论在训练还是预测过程中，模型都存在随机性**，相比之下，大部分带有随机性的算法只会在训练的过程中有随机性，而在测试过程中对模型进行固定。但由于变分自动编码器的“随机性”是与网络架构及输入数据结构都高度相关的随机性，因此当训练数据变化的时候，随机抽样的情况也会跟着变化。

2. **可以作为生成模型使用**。其他的自动编码器都是在原始图像上进行修改，而变分自动编码器可以从无到有生成与训练集高度相似的数据。由于输入Decoder的信息只是从正态分布中抽选的随机样本，因此其本质与随机数差异不大，当我们训练完变分自动编码器之后，就可以只使用解码器部分，只要对解码器输入结构正确随机数/随机矩阵，就可以生成与训练时所用的真实数据高度类似的数据。

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/01/ML1533-image003.jpg" height = 200>

- **VAE的损失函数**

那这样一个架构的损失函数如何表示呢？变分自动编码器的损失函数是整个架构中的一大难点，其损失函数的推导过程要求我们对概率论、信息论等数学知识有很深的了解，但如果光是解释损失函数本身，其实并没有那么困难。来看，以下是变分自动编码器论文当中所提供的损失函数的公式：

$$ L(\theta, \phi;\boldsymbol{x,z}) = \mathbb E_{z \sim  {q_{\phi}\ \boldsymbol{(x|z)}}}\  [\log \color{magenta}{\boldsymbol{p_{\theta}(\boldsymbol{x|z})}}] - D_{KL} \left(\color{blue}{\boldsymbol{q_{\phi}(\boldsymbol{z|x})}} \  || \ \color{red}{\boldsymbol{ p(\boldsymbol{z})}} \right)
$$

这一公式符号繁多、眼花缭乱，我们一个个元素来拆解——

> 公式中的$p$与$q$都是**分布**，一组数据的分布可以由数据本身来表示，也可以由当前数据的均值和标准差来表示。在当前公式当中，两种表示方法我们都有用到。<br><br>
> $\theta$, $\phi$是**自动编码器要求解的参数**，其中$\phi$是编码器Encoder上各个线性层/卷积层/其它层的参数，$\theta$是解码器Decoder上各个线性层/卷积层/其它层的参数。<br><br>
> $x$，$z$是**输入架构的数据**，$x$是输入编码器Encoder的原始数据，$z$是输入解码器Decoder的原始数据。

了解这些基本信息后，再来看损失函数公式中被重点突出的部分：

- $\color{red}{\boldsymbol{ p(\boldsymbol{z})}}$：$z$的分布。在整个变分自动编码器中，所有的$z$都是从正态分布中抽样出来的，因此$z$的分布就是完美正态分布，也就是之前我们提到的`Target Distribution`目标分布。

- $\color{blue}{\boldsymbol{q_{\phi}(\boldsymbol{z|x})}}$：在知晓$x$的条件下，以$\phi$为参数推断出的$z$的分布，即以$x$为输入，以$\phi$为参数推断出的$z$的具体数据。不难发现，这一过程就是Encoder的过程：因此$\color{blue}{\boldsymbol{q_{\phi}(\boldsymbol{z|x})}}$的本质就是Encoder输出层输出的那些均值和标准差，他们代表了我们之前提到的`Actual Distribution`。

- $\color{magenta}{\boldsymbol{p_{\theta}(\boldsymbol{x|z})}}$：在知晓$z$的条件下，以$\theta$为参数推断出的$x$的分布，即以$z$作为输入，以$\theta$作为参数而推断出的$x$的具体数据。不难发现，这一过程就是Decoder的过程，所以$\color{magenta}{\boldsymbol{p_{\theta}(\boldsymbol{x|z})}}$实际上是直接指向Decoder的输出。而在公式前的脚标中，特地标注了数据$z$的来源，不难发现，$z \sim  {q_{\phi}\ \boldsymbol{(x|z)}}$说明$z$是Encoder部分输出的结果，更加佐证了$\color{magenta}{\boldsymbol{p_{\theta}(\boldsymbol{x|z})}}$是decoder过程的结果。

将这三个元素拆解后，我们的损失函数可以被改写成：

$$ L(\theta, \phi;\boldsymbol{x,z}) = \mathbb E_q[\log (\color{magenta}{Decoder的输出分布})] - D_{KL} \left(\color{blue}{Encoder的输出分布} \  || \ \color{red}{预设的正态分布} \right)
$$

在这样的状态下，再来解读这一损失函数就容易多了。

> - 损失函数的后半部分

先来看后半部分，这是一个KL散度的计算公式，在原始论文当中被称之为“隐式损失”（Latent Loss）。KL散度是衡量两组数据分布差异的衡量指标，也是衡量分布A在变化成分布B过程中损失的信息量的指标，因此当两组数据的分布越接近时，KL散度就会越小，反之KL散度会越大。

在我们的损失函数当中，很明显KL散度衡量的是Encoder的输出与预设的正态分布之间的差异，这说明**损失函数希望Encoder输出的结果越接近正态分布越好**，因此在最初介绍自动变分编码器流程时，我们才会认为“变分自动编码器中的编码器会尽量将样本$i$所携带的所有特征信息$X_i$的分布转码成**类高斯分布**$d_i$”。

这一过程其实并不难理解：在变分自动编码器的Encoder中，我们从原始数据上推断出均值与标准差，并且用这些均值和标准差构筑正态分布，再从正态分布中抽取样本输入Decoder。毫无疑问的，当Encoder输出的数据分布越接近正态分布时，我们所构筑的正态分布才会越靠近原始数据中的信息，从这样的正态分布中抽取的样本才会更接近真实的数据样本。因此KL散度是为了逼迫Encoder向着正态分布方向解码原始数据而存在的，损失函数中的惩罚项。

一般来说，当我们将从样本生成的均值与标准差带入后，KL散度可以写作：

$$KL_i = -\frac{1}{2}\sum^K_{j=1} \left(1+log(\sigma_j^2) - \sigma_j^2 -\mu_j^2 \right)$$

这就是我们在实际执行代码时所写的公式。其中$K$指的是对一个样本生成了$K$组均值和标准差，$j$指的是当前均值和标准差的具体组数，对任意样本$i$，我们需要将全部的$K$组均值和标准差进行加和后计算。

> - 损失函数的前半部分

$$ L(\theta, \phi;\boldsymbol{x,z}) = \mathbb E_q[\log (\color{magenta}{Decoder的输出分布})] - D_{KL} \left(\color{blue}{Encoder的输出分布} \  || \ \color{red}{预设的正态分布} \right)
$$

现在我们已经了解了损失函数的后半部分了，那它的前半部分是什么呢？虽然无法从肉眼上明显地看出来对Decoder的输出分布求对数是怎样的含义，但变分自动编码器的终极目标依然是输出与原始数据高度相似的数据，因此变分自动编码器的损失函数中必然包含重构损失Reconstruction Loss这一衡量输入与输出差异的部分。因此很明显，log(Decoder输出的分布)就是重构损失。这一形式有些类似于二分类交叉熵中所表示的$ylogp(x)$，只不过我们现在是无监督算法，并无真实标签罢了。在实际的代码执行过程中，我们一般使用MSE或者二分类交叉熵损失的均值来替代上述公式。

因此，真正在反向传播中使用的损失函数是：

$$L(\theta, \phi) = \frac{1}{m}\sum_{i=1}^M(x_i - \hat{x_i})^2 - \frac{1}{2m}\sum_{i=1}^M\sum_{j=1}^K \left(1+log(\sigma_j^2) - \sigma_j^2 -\mu_j^2 \right)$$

- **重参数化技巧**（reparameterization trick）

现在我们已经了解了变分自动编码器的基本流程和训练目标，那我们是否可以尝试着手动去实现变分自动编码器了呢？有一个隐藏的陷阱还没有被我们注意到，那就是**由于抽样流程的存在，架构中的数据流是断裂的，因此反向传播无法进行**。反向传播要求每一层数据之间必有函数关系，而抽样流程不是一个函数关系，因此无法被反向传播。为了解决这一问题，变分自动编码器的原始论文提出了**重参数化技巧，这一技巧可以帮助我们在抽样的同时建立$z$与$\mu$和$\sigma$之间的函数关系**，这样就可以令反向传播顺利进行了。

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/01/ML1533-image003.jpg" height = 200>

先来复习一下原本的抽样流程：对任意样本$i$，我们首先需要从Encoder处获得$(\mu_i,\sigma_i)$，然后基于这两个元素构建正态分布，之后从构建好的正态分布中进行抽样。如果使用简单的numpy代码来表示这个过程，则有：

In [2]:
import numpy as np

#获取三组mu与sigma
mu = [3,2,5]
sigma = [5,4,7]

#以mu和sigma为基准构建三组正态分布
np.random.seed(420)
d1 = np.random.normal(loc=mu[0],scale=sigma[0],size=(100,))
d2 = np.random.normal(loc=mu[1],scale=sigma[1],size=(100,))
d3 = np.random.normal(loc=mu[2],scale=sigma[2],size=(100,))

#抽样

d1[50]

-1.316504668771417

In [3]:
d2[2]

-0.6201637857596833

In [4]:
d3[87]

0.382754792922813

In [5]:
z = [d1[50],d2[2],d3[87]]

In [6]:
z

[-1.316504668771417, -0.6201637857596833, 0.382754792922813]

抽出的三个数字最终组合为$z$被输入Decoder。在这个流程中，我们如何才能够建立$\sigma$、$\mu$、z之间的函数关系呢？答案是利用**0-1标准化**。0-1标准化是一个非常简单的操作，普通的正态分布$N \sim (\mu,\sigma)$被0-1标准化后都可以变为均值为0，标准差为1的标准正态分布$N \sim (0,1)$。假设$z$满足任意正态分布，$z'$是0-1标准化后的结果，则有：

$$\begin{aligned}
z' &= \frac{z - \mu}{\sigma}
\end{aligned}
$$

不难发现，当我们知道$z$而求解$z'$时，这一等式就是$z'$与均值标准差之间的函数关系，因此我们也可以轻松地逆转这个操作（“范标准化”），让上述式子变成$z$与$\sigma$和$\mu$之间的函数关系：

$$\begin{aligned}
z' &= \frac{z - \mu}{\sigma} \\ z &= z' * \sigma + \mu
\end{aligned}
$$

如此，我们就建立了$z$与$\sigma$和$\mu$之间的关联，并且这个式子与变分自动编码器中的流程十分契合：我们总是先知道$\mu$和$\sigma$，才能够求解出具体的$z$的。但是，此时这个函数关系中多出了一个在变分自动编码器中不曾存在的分布：标准正态分布$z'$，同时抽样过程也不复存在，我们应该如何将之前的流程与该函数关系结合呢？答案是转变思路，**先从标准正态分布中抽样出$z'_i$、再使用抽样出的结果、固定的均值和标准差计算出**$z_i$。

在数学上，从$N \sim (\mu,\sigma)$直接抽样$z_i$的行为**完全等同于**从标准正态分布$N \sim (0,1)$中抽样$z'$后、再对其进行反标准化的行为。我们可以使用代码来验证一下：

In [ ]:
(mu, sigma)  -   N(mu,sigma)   - 抽样 z[50]

N(0,1) - z'[50] - z'[50] * sigma + mu

In [7]:
#获取mu与sigma
mu =    [3,2,5]
sigma = [5,4,7]

#建立三个标准正态分布
np.random.seed(420)
sd1 = np.random.normal(loc=0,scale=1,size=(100,))
sd2 = np.random.normal(loc=0,scale=1,size=(100,))
sd3 = np.random.normal(loc=0,scale=1,size=(100,))

#建立z与mu和sigma之间的函数关系
#求解z
z1 = sd1[50] * sigma[0] + mu[0]
z1

-1.316504668771417

In [8]:
z2 = sd2[2] * sigma[1] + mu[1]
z2

-0.6201637857596833

In [9]:
z3 = sd3[87] * sigma[2] + mu[2]
z3

0.382754792922813

In [10]:
#根据sigma和mu建立正态分布后再分别抽样，得出的结果
z

[-1.316504668771417, -0.6201637857596833, 0.382754792922813]

不难发现，两种操作得出的数据完全一致，但在第二种操作当中，对标准正态分布进行抽样后，只需要向函数$z$提供抽样出的基本数据，就可以让$\mu$和$\sigma$被当做函数z的参数参与到反向传播的计算当中，而对标准正态分布进行抽样的这一无法使用函数表达的行为，就被排除在了计算图之外。如此，就可以对变分自动编码器进行反向传播了。

![](https://nbisweden.github.io/workshop-neural-nets-and-deep-learning/session_rAutoencoders/lecture_autoencoders_r/assets/VAE_reparam.png)

#### 4.5.3 图像分割经典架构Unet

- **图像分割必备的基础知识**

图像分割（image Segmentation）是深度学习在图像领域的重要应用之一，如果说识别任务是针对一张图像进行的学习、检测任务是针对图像中不同的对象进行的学习，那分割任务就是针对单一像素进行的学习。分割任务是像素级别的有监督任务，在图像分割时，我们需要对图像中的每一个像素进行分类，因此我们可以找出图像中每个对象的“精确边界”。以下面的图像为例，我们可以找出["猫","狗"]两个标签类别所对应的具体对象，并且找出这些对象的精确边界。当原始图像越复杂，分割任务中需要输出的标签类别也就越多，对下面的图像，除了也可以["猫","狗"]分割之外，还可以对["草坪","项圈"]等标签进行分割。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/40.jpg)

从分割的类别来看，我们可以执行将不同性质的物体分开的语义分割（semantic segmentation），也可以执行将每个对象都分割开来的实例分割（instance segmentation），还可以执行使用多边形或颜色进行分割的分割方法。同时，根据分割的“细致程度”，还可以分为粗粒度分割（Coarse Segmentation）与细粒度分割（Fine Segmentation），只要拥有对应的标签，我们就可以将图像分割到非常非常精细的程度：
![image.png](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/39.jpg)
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/41.jpg)

不难发现，分割任务具体可以做到的分割程度是由训练图像中的标签决定的，而分割图像中的标签具体是什么样呢？在分割任务中，训练数据是原始图像，原始图像中存在的所有对象都可以被标记为某一类别，而一张图像所对应的标签一般是与原始图像相同尺寸的标签矩阵，该矩阵被上色之后被称为“遮罩矩阵”(mask)。通常来说，遮罩矩阵中的颜色数量等同于这张标签上的标签类别数量。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/52.png?versionId=CAEQKBiBgIDOufmqkBgiIGYzZjBiYWVmOGVkYjQzZDZiM2QwYTVlY2E0YjZmYWFk)

以下面最为简单的10x10尺寸图像为例，假设一张原始图像（一个样本）的结构为(1,10,10)，那它所对应的标签的结构一般也为(1,10,10)，假设总共有n_samples个样本，则数据集的标签格式为(n_samples,1,10,10)。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/41_.png)

当输入标签为图像/矩阵时，你能推断出分割任务中网络对应的输出是什么吗？对于**任意用于图像识别的数据集**，如果数据有num_classes个标签类别，则**神经网络会对每个样本输出num_classes个对应的概率**，此时作为输出层的线性层则会有num_classes个神经元，作为输出层的卷积层则会输出num_classes个1x1的特征图。因此不难推断，**在分割任务中**，如果数据包含num_classes个标签类别，**神经网络则需要对每个像素值输出num_classes个对应的概率**。因此，分割任务中一个样本所对应的输出是该样本上所有像素值、在所有标签类别下的概率。具体来看，以10x10的花朵图为例，该数据集的标签有6大类别，因此一个样本所对应的输出就有6张概率图，一张概率图对应着一个类别，反馈着样本上所有的像素是0类、1类、2类……num_classes类的概率。**当输出概率图后，一个像素在所有概率图上最大的概率所对应的类别，就是该像素的预测类别**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/42.png)

需要注意的是，以上图像中有不严谨之处。对一个像素而言，该像素为任意类别的概率加和之后应该为1（例如，对图像上最右下角的像素点而言，所有标位灰色的概率值加和之后应该为1）。在绘制图像时，受限于绘图公式设计，图像中并没有实现“单一像素的所有类别概率加和必须为1”这一条件，但在实际使用数据、输出结果是，这一条件是一定会被满足的。

同时，还有几个值得注意的问题：

1. **一张图上的存在的标签类别可能少于整个数据集中的标签类别**。标签类别数量会覆盖整个数据集上的标签类别，因此一张图像上不一定包含了所有的标签类别，但每个样本都必须输出所有类别的概率图。假设一张图像上没有类别A，那我们会期待这张图像所对应的类别A的概率图上的值会全部为0，即没有任何像素属于该类别。

2. **由于需要输出概率图，所以分割网络的输出层一般都是卷积层**。以此为基础还诞生了整个网络中只有卷积或卷积相关计算的网络全卷积网络FCN（fully convolutional network）。

3. **虽然标签和输出都转化为了图像，但我们常用的交叉熵损失等损失函数依然可以使用**，只不过此时我们公式中的预测值是多个矩阵，真实标签也是多个矩阵。具体来看：

> **二分类交叉熵损失**——<br><br>
>$$L = -\left( y\log p(x) + (1 - y)\log(1 - p(x)) \right)$$<br>
>此时y就是标签矩阵，而$p(x)$就是对应的概率矩阵。<br><br>
> **多分类交叉熵损失**，总共有K个类别——<br><br>
> $$L = -\sum_{k=1}^Ky^*_k\log(P^k(x))$$<br>
> 在标签为序列的识别任务中，$P^k(x)$是一个样本的类别为k的概率，并且$p^k(x) = Softmax(网络输出)$，$y^*$是由真实标签做独热编码后的向量。例如，在3分类情况下，真实标签$y_i$为2时，$y^*$为[$y^*_{1}$, $y^*_{2}$, $y^*_{3}$]，取值分别为：<br>
> 
> |$y^*_{1}$|$y^*_{2}$|$y^*_{3}$|
> |:-:|:-:|:-:|
> |$0$|$1$|$0$|
> 
> 而在标签为概率图的分割任务中，$P^k(x)$是所有样本的类别为k的概率（即概率图），并且$y^*$是将标签矩阵转化为独热形态后的独热矩阵。以之前的花朵图像为例，在num_classes个标签分类的情况下，我们会分割出num_classes个独热矩阵，如果一个像素的真实标签为该类别，则该像素在这张独热矩阵上的标签为1，否则标签为0。<br>
> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/43.png)

4. **标签与输出概率图的尺寸必须一致，但有时候标签和输出概率图的尺寸可以略小于原始图像**。

> 图像分割是对每个样本上的每个像素进行分类，因此最为严谨的情况下每个像素都应该有对应的标签，但对像素级别进行分割的根本目的是为了描绘出物体的轮廓。然而，当原始图像的尺寸很大、像素很高时，我们并不需要非常高清的图像才能够展示出物体的轮廓；同时，在许多时候我们需要的只是“大致轮廓”，而不需要非常精确的边缘，因此标签图像可以略小于原始图像。这样的标签可以反馈出每一类对象在原始图像中的大致轮廓，但不能精确地反馈出原始图像的每一个像素点的类别。<br>
> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/52.png?versionId=CAEQKBiBgIDOufmqkBgiIGYzZjBiYWVmOGVkYjQzZDZiM2QwYTVlY2E0YjZmYWFk)<br><br>
> 标签需要被放置到损失函数中使用，为了能够和标签图像进行像素值一一对应的计算，分割架构输出的概率图尺寸必须与标签尺寸一致。因此当标签小于原始图像时，概率图也会小于原始图像。并且，标签和概率图越小，其投射到原始图像上的轮廓就越不准确，依据我们的使用场景，我们可以调整网络架构的输出：

较为粗糙的分割<br>（概率图/标签可以大幅度小于原始图像）            |  更加精准地分割<br>（概率图/标签可以略小于原始图像）
:-------------------------:|:-------------------------:
<img src="http://149695847.v2.pressablecdn.com/wp-content/uploads/2021/02/instance-output.png" alt="drawing" width="1000"/>|<img src="https://149695847.v2.pressablecdn.com/wp-content/uploads/2021/02/panoptic-output.png" alt="drawing" width="1000"/>

现在你已经了解了关于图像分割架构的一些基本特点了。有了这些基础知识，我们在解析各种分割架构的时候就不容易进入迷雾之中，下一节我们将展开聊聊经典分割架构Unet的结构。

- **Unet**

Unet是一个博采众长的架构，它于2015年被德国弗莱堡大学的研究团队提出，并在原始论文当中被用于生物医学影像的分割。但事实上，它既可以完成分类任务，也可以完成分割任务，还可以被当做无监督算法使用，这是因为它汲取了大量其他网络的精华结构、并且有机地将这些架构融合在了一起。具体地来看，Unet的架构图如下：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/53.png)

整个架构图呈现对称的“U字型”，因此该网络被称为Unet。查看架构细节，不难发现Unet其实拥有多重身份和多重标签：

1. **Unet是一个全卷积网络（FCN）**，它没有使用除了卷积、池化和转置卷积之外的任何层。

2. **Unet是一个Encoder-Decoder**，它的结构与深度卷积自动编码器高度相似：输入大图像、使用卷积层和池化层组成的Encoder将图像压缩，形成数据表示结果codes，接着又使用由卷积层和转置卷积层组成的Decoder将图像放大，最终输出大图像。这一结构左右对称，与自动编码器非常类似。

3. **Unet是一个分割网络**，因此它所使用的标签是遮罩，要输出的是概率图，使用的损失函数是二分类交叉熵损失。从最终输出的结果来看，Unet被创造的原始论文中所使用的分割图像应该是二分类的分割图像。同时，网络输入图像尺寸与输出图像尺寸差异较大，因此可以判断原始论文中所使用的生物医学影像并不需要太高的轮廓精度，检查原始论文，会进一步印证我们的观点：
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/44.png)

4. **Unet使用了跳跃链接**，直接将Encoder中尚未压缩完毕的图像传向Decoder，这样可以将更多原始信息直接传向输出方向，帮助生成与原始数据更相似的图像。经过跳跃链接穿到Decoder的图像需要与经过编码后的数据合并，再进行卷积，这样Decoder在输出最终的概率图时可以参考的信息就变得更加丰富了。

接下来，让我们来复现一下Unet架构：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/53.png)

在深度学习的后期，许多时候架构中的具体层已经不是最为关键的内容了，相对的，**数据如何在架构中流通才是我们真正关心的内容**。观察架构图，我们可以将Unet总结为如下结构：

输入 → 【双卷积 + 池化】x4 → 【双卷积】 → 【转置卷积 + 双卷积】x4 → 【1x1卷积】→ 输出<br>
**&emsp;&emsp;&emsp; [------Encoder------] &emsp;[bottleneck]&emsp;[-------Decoder-------]&emsp;&emsp;&emsp;[Output]**

因此毫无疑问的，我们可以先定义一个双卷积的结构。在原始论文中规定，无论是在Encoder还是Decoder当中，每个卷积层后面都跟ReLU激活函数。同时，作为卷积架构的惯例，我们在每个卷积层后面、ReLU激活函数之前使用Batch Normalization。需要注意的是，Unet架构中使用的不是令特征图尺寸保持不变的卷积层，而是每经过一个卷积就会将特征图长宽缩小2的卷积层。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/50.png)

In [31]:
class DoubleConv2d(nn.Module):
    def __init__(self,in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels,3,1,0,bias=False)
                                 ,nn.BatchNorm2d(out_channels)
                                 ,nn.ReLU(inplace=True)
                                 ,nn.Conv2d(out_channels, out_channels,3,1,0,bias=False)
                                 ,nn.BatchNorm2d(out_channels)
                                 ,nn.ReLU(inplace=True)
                                 )
    def forward(self,x):
        return self.conv(x)

接下来我们来考虑如何实现该架构中的数据流。Unet中的数据并不依照从左到右进行线性流动：**因为有跳跃链接的存在，Encoder中的每个双卷积的输出结果都必须被直接传输到Decoder中每个双卷积的输入层**。因此每经过一次双卷积结构我们就需要保存中间结果，因此我们可以按如下方式梳理数据流：

In [ ]:
#Encoder
#输入x

#x = 双卷积(x)
#保存x1
#x = 池化层(x)

#x = 双卷积(x)
#保存x2
#x = 池化层(x)

#x = 双卷积(x)
#保存x3
#x = 池化层(x)

#x = 双卷积(x)
#保存x4
#x = 池化层(x)

In [ ]:
#完全等同于

l = []

for i in range(4):
    #x = 双卷积(x)
    #l.append(x)
    #x = 池化层(x)

#在这个循环中，4个卷积层的输入特征图数量和输出特征图数量不一致，因此实际上是4个不同的双卷积结构
#但相对的，池化层是完全一致的Maxpool(2)，且池化层没有需要迭代的权重，因此可以被重复使用
#所以，我们可以将4个卷积层定义成一个序列，并单独定义池化层

for i in range(4):
    #x = 卷积层序列[i](x)
    #l.append(x)
    #x = 池化层(x)
    
#l = [x1,x2,x3,x4]

相似地，**由于跳跃链接的存在，我们需要将数据与跳跃链接传过来的数据合并后，才能输入到Decoder中的每个双卷积层**。并且需要注意的是，在Encoder中池化层是位于双卷积的后面，但在Decoder中转置卷积层是位于双卷积层的前面：

输入 → 【双卷积 + 池化】x4 → 【双卷积】 → 【转置卷积 + 双卷积】x4 → 【1x1卷积】→ 输出<br>
**&emsp;&emsp;&emsp; [------Encoder------] &emsp;[bottleneck]&emsp;[-------Decoder-------]&emsp;&emsp;&emsp;[Output]**

In [ ]:
#Decoder
#l
#x此时是瓶颈结构的输出

#x = 转置卷积(x)
#x = 合并(x,x4)
#x = 双卷积(x)

#x = 转置卷积(x)
#x = 合并(x,x3)
#x = 双卷积(x)

#x = 转置卷积(x)
#x = 合并(x,x2)
#x = 双卷积(x)

#x = 转置卷积(x)
#x = 合并(x,x1)
#x = 双卷积(x)

In [ ]:
#按循环写：

for i in range(4):
    #x = 转置卷积[i](x)
    #x = 合并(x,l[i])
    #x = 双卷积[i](x)

#其中4个转置卷积的参数不同，4个双卷积的参数也不同，因此都需要构建成序列

- Unet架构复现

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/HealthCareProject/53.png)

In [35]:
class Unet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder_conv = nn.Sequential(DoubleConv2d(1,64)
                                          ,DoubleConv2d(64,128)
                                          ,DoubleConv2d(128,256)
                                          ,DoubleConv2d(256,512)
                                         )
        
        self.encoder_down = nn.MaxPool2d(2)
        
        self.decoder_up = nn.Sequential(nn.ConvTranspose2d(1024,512,4,2,1)
                                       ,nn.ConvTranspose2d(512,256,4,2,1)
                                       ,nn.ConvTranspose2d(256,128,4,2,1)
                                       ,nn.ConvTranspose2d(128,64,4,2,1)
                                       )
        
        self.decoder_conv = nn.Sequential(DoubleConv2d(1024,512)
                                          ,DoubleConv2d(512,256)
                                          ,DoubleConv2d(256,128)
                                          ,DoubleConv2d(128,64)
                                         )
        
        self.bottleneck = DoubleConv2d(512,1024)
        
        self.output = nn.Conv2d(64,2,3,1,1)
    
    def forward(self,x):
        
        #encoder：保存每一个DoubleConv的结果为跳跃链接做准备，同时输出codes
        skip_connection = []
        
        for conv in self.encoder_conv:
            x = conv(x)
            skip_connection.append(x)
            x = self.encoder_down(x)
        
        x = self.bottleneck(x)
        
        #调换顺序
        skip_connection = skip_connection[::-1]
        
        #decoder：codes每经过一个转置卷积，就需要与跳跃链接中的值合并
        #合并后的值进入DoubleConv
        
        for idx in range(4):
            x = self.decoder_up[idx](x)
            skip_connection[idx] = transforms.functional.resize(skip_connection[idx],size=x.shape[-2:])
            x = torch.cat((skip_connection[idx],x),dim=1)
            x = self.decoder_conv[idx](x)
        
        x = self.output(x)
        return x

In [ ]:
#架构验证
net = Unet()
summary(net,input_size=(10,1,572,572),device="cpu")